# Init

In [114]:
# Import utils
import numpy as np
import pandas as pd
import math
import time
import json
import pyreadr
import pickle
from joblib import dump, load, Parallel, delayed
import os
import copy
import datetime as dt
from tqdm import tqdm


# Import ML models
import sklearn
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit

from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

from dddex.levelSetKDEx_univariate import LevelSetKDEx, LevelSetKDEx_NN
from dddex.wSAA import RandomForestWSAA, SampleAverageApproximation

# Weights & Biases
import wandb

# Import Gurobi
import gurobipy as gp
from gurobipy import GRB

# Optimization Module
from DataDrivenPatientScheduling import WeightsModel
from DataDrivenPatientScheduling import Experiment

In [115]:
# Setup the experiment
experiment_setup = dict(

    # Paths
    path_data = '/home/fesc/dddex/PatientScheduling/Data',
    path_models = '/home/fesc/dddex/PatientScheduling/Data/Models',
    path_results = '/home/fesc/dddex/PatientScheduling/Data/Results',

    ## Optimization
    optimization_params = dict(
    
        # Gurobi params
        gurobi_params = {

            'LogToConsole': 0, 
            'Threads': 2

        },

        # Cost params
        cost_params = [

            {'CR': 0.10, 'c_waiting_time': 1, 'c_overtime': 9},
            {'CR': 0.25, 'c_waiting_time': 1, 'c_overtime': 3},
            {'CR': 0.50, 'c_waiting_time': 1, 'c_overtime': 1},
            {'CR': 0.75, 'c_waiting_time': 3, 'c_overtime': 1},
            {'CR': 0.90, 'c_waiting_time': 9, 'c_overtime': 1}

        ],

        # Number of scenarios
        K = 10**3,
        
        # Time budget multiplier
        rho = 1,

        # n parallel jobs
        n_jobs = 32
        
    )
)

# Make all experiment variables visible locally
locals().update(experiment_setup)

# Model LSx - LGBM

In [117]:
# Setup the model
model_setup = dict(

    # Model
    model_name = 'LSx_LGBM',
        
    ## Point estimator
    point_estimator_params = dict(

        # Meta parameters
        model_params = {
            'random_state': 12345,
            'n_jobs': 4,
            'verbose': -1
        },

        # Tuning meta params
        tuning_params = {     
            'n_iter': 200,
            'scoring': {'MSE': 'neg_mean_squared_error'},
            'return_train_score': True,
            'refit': 'MSE',
            'random_state': 12345,
            'n_jobs': 8,
            'verbose': 0
        },    

       # Hyper params search grid
       hyper_params_grid = {
            'num_leaves': [x for x in range(5, 500, 5)],
            'max_depth': [-1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            'min_child_samples': [x for x in range(5, 500, 5)],
            'learning_rate': [x/100 for x in range(5, 25, 5)],
            'n_estimators': [100, 200, 300, 400, 500],
            'subsample': [x/100 for x in range(5, 100, 5)],
            'colsample_bytree': [x/100 for x in range(5, 100, 5)]
        },  
    
    
    ),
          
 
    ## Density estimator
    density_estimator_params = dict(
    
        # Meta parameters
        model_params = {
            'weightsByDistance': False
        },

        # Tuning meta params
        tuning_params = {     
            'probs': [0.005, 0.025, 0.165, 0.250, 0.500, 0.750, 0.835, 0.975, 0.995],
            # 'probs': sorted(list(set(np.concatenate([np.array([0.005, 0.025, 0.165, 0.25, 0.50, 0.835, 0.75, 0.975, 0.995]), 
            #                                          np.arange(1, 100, 1) / 100])))),
            'n_jobs': 8,
        },    

        # Hyper params search grid
        hyper_params_grid = {
            'binSize': [x for x in range(10, 500, 10)]
        },
        
    )
)

# Make all model variables visible locally
locals().update(model_setup)

## Preprocessing

In [118]:
# Load data
Y_data = pd.read_csv('/home/fesc/dddex/PatientScheduling/Data/Y_data.csv')
X_data = pd.read_csv('/home/fesc/dddex/PatientScheduling/Data/X_data.csv')
ID_data = pd.read_csv('/home/fesc/dddex/PatientScheduling/Data/ID_data.csv')

In [119]:
# Train-test split
y_train = np.array(Y_data.loc[ID_data.train_test == 'train']).flatten()
X_train = np.array(X_data.loc[ID_data.train_test == 'train'])
ID_train = ID_data.loc[ID_data.train_test == 'train']

In [120]:
# CV folds
tscv = TimeSeriesSplit(n_splits=3)
cv_folds = tscv.split(range(len(ID_train)))
cv_folds = list(cv_folds)

In [121]:
# Initialize modules
weightsmodel = WeightsModel()
experiment = Experiment()

## Tune point estimator

In [122]:
# Update params
locals().update(point_estimator_params)

# Point estimator
point_estimator = LGBMRegressor(**model_params)

# Tune point estimator
point_estimator = weightsmodel.tune_point_estimator(X_train, y_train, point_estimator, cv_folds, hyper_params_grid, 
                                                    tuning_params, random_search=True, print_time=True)

Time: 0:00:08
>> Execution time: 8.0 seconds
>> CPU time: 1.0 seconds


## Tune density estimator

In [123]:
# Update params
locals().update(density_estimator_params)

# Density estimator
density_estimator = LevelSetKDEx(estimator = point_estimator, **model_params)

# Tune density estimator
density_estimator = weightsmodel.tune_density_estimator(X_train, y_train, density_estimator, cv_folds, hyper_params_grid, 
                                                        tuning_params, random_search=False, print_time=True)

# Save
save = dump(density_estimator, path_models+'/density_estimator_'+model_name+'.joblib')

Time: 0:00:03
>> Execution time: 3.0 seconds
>> CPU time: 0.0 seconds


## Data-driven optimization

In [124]:
# Update params
locals().update(optimization_params)

# Load density estimator
density_estimator = load(path_models+'/density_estimator_'+model_name+'.joblib')

# Test dates
dates = pd.Series(list(set(ID_data.loc[ID_data.train_test == 'test', 'date']))).sort_values()

# Timer
start_time, st_exec, st_cpu = dt.datetime.now().replace(microsecond=0), time.time(), time.process_time()      
        
# For each date in the test horizon
with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(dates))) as progress_bar:
    results = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(
        X = np.array(X_data),
        y = np.array(Y_data),
        date = date,
        dates = ID_data['date'],
        areas = ID_data['area'],
        weightsModel = density_estimator,
        cost_params = cost_params,
        gurobi_params = gurobi_params,
        K = K,
        rho = rho,
        print_status = False) for date in dates)
    
# Finalize
results = pd.concat(results).reset_index(drop=True)

# Save results
results.to_csv(path_results+"/"+model_name+"_K"+str(K)+".csv", sep=',', index=False)

# Time
print('Time:', dt.datetime.now().replace(microsecond=0) - start_time)  
print('>> Execution time:', np.around(time.time()-st_exec, 0), "seconds") 
print('>> CPU time:', np.around(time.process_time()-st_cpu, 0), "seconds")

Progress: 100%|██████████| 64/64 [01:02<00:00,  1.03it/s]

Time: 0:01:02
>> Execution time: 62.0 seconds
>> CPU time: 3.0 seconds


# Model: LSx NN - LGBM

In [125]:
# Setup the model
model_setup = dict(
    
    # Model
    model_name = 'LSx_LGBM_NN',
        
    ## Point estimator
    point_estimator_params = dict(

        # Meta parameters
        model_params = {
            'random_state': 12345,
            'n_jobs': 4,
            'verbose': -1
        },

        # Tuning meta params
        tuning_params = {     
            'n_iter': 200,
            'scoring': {'MSE': 'neg_mean_squared_error'},
            'return_train_score': True,
            'refit': 'MSE',
            'random_state': 12345,
            'n_jobs': 8,
            'verbose': 0
        },    

        # Hyper params search grid
       hyper_params_grid = {
            'num_leaves': [x for x in range(5, 500, 5)],
            'max_depth': [-1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            'min_child_samples': [x for x in range(5, 500, 5)],
            'learning_rate': [x/100 for x in range(5, 25, 5)],
            'n_estimators': [100, 200, 300, 400, 500],
            'subsample': [x/100 for x in range(5, 100, 5)],
            'colsample_bytree': [x/100 for x in range(5, 100, 5)]
        },  
    
    
    ),
          
 
    ## Density estimator
    density_estimator_params = dict(
    
        # Meta parameters
        model_params = {
            'weightsByDistance': True
        },

        # Tuning meta params
        tuning_params = {     
            'probs': [0.005, 0.025, 0.165, 0.250, 0.500, 0.750, 0.835, 0.975, 0.995],
            # 'probs': sorted(list(set(np.concatenate([np.array([0.005, 0.025, 0.165, 0.25, 0.50, 0.835, 0.75, 0.975, 0.995]), 
            #                                          np.arange(1, 100, 1) / 100])))),
            'n_jobs': 8,
        },    

        # Hyper params search grid
        hyper_params_grid = {
            'binSize': [x for x in range(10, 500, 10)]
        },
        
    )
)

# Make all model variables visible locally
locals().update(model_setup)

## Preprocessing

In [126]:
# Load data
Y_data = pd.read_csv('/home/fesc/dddex/PatientScheduling/Data/Y_data.csv')
X_data = pd.read_csv('/home/fesc/dddex/PatientScheduling/Data/X_data.csv')
ID_data = pd.read_csv('/home/fesc/dddex/PatientScheduling/Data/ID_data.csv')

In [127]:
# Train-test split
y_train = np.array(Y_data.loc[ID_data.train_test == 'train']).flatten()
X_train = np.array(X_data.loc[ID_data.train_test == 'train'])
ID_train = ID_data.loc[ID_data.train_test == 'train']

In [128]:
# CV folds
tscv = TimeSeriesSplit(n_splits=3)
cv_folds = tscv.split(range(len(ID_train)))
cv_folds = list(cv_folds)

In [129]:
# Initialize modules
weightsmodel = WeightsModel()
experiment = Experiment()

## Tune point estimator

In [130]:
# Update params
locals().update(point_estimator_params)

# Point estimator
point_estimator = LGBMRegressor(**model_params)

# Tune point estimator
point_estimator = weightsmodel.tune_point_estimator(X_train, y_train, point_estimator, cv_folds, hyper_params_grid, 
                                                    tuning_params, random_search=True, print_time=True)

Time: 0:00:08
>> Execution time: 9.0 seconds
>> CPU time: 1.0 seconds


## Tune density estimator

In [131]:
# Update params
locals().update(density_estimator_params)

# Density estimator
density_estimator = LevelSetKDEx_NN(estimator = point_estimator, **model_params)

# Tune density estimator
density_estimator = weightsmodel.tune_density_estimator(X_train, y_train, density_estimator, cv_folds, hyper_params_grid, 
                                                        tuning_params, random_search=False, print_time=True)

# Save
save = dump(density_estimator, path_models+'/density_estimator_'+model_name+'.joblib')

Time: 0:00:21
>> Execution time: 21.0 seconds
>> CPU time: 0.0 seconds


## Data-driven optimization

In [132]:
# Update params
locals().update(optimization_params)

# Load density estimator
density_estimator = load(path_models+'/density_estimator_'+model_name+'.joblib')

# Test dates
dates = pd.Series(list(set(ID_data.loc[ID_data.train_test == 'test', 'date']))).sort_values()

# Timer
start_time, st_exec, st_cpu = dt.datetime.now().replace(microsecond=0), time.time(), time.process_time()      
        
# For each date in the test horizon
with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(dates))) as progress_bar:
    results = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(
        X = np.array(X_data),
        y = np.array(Y_data),
        date = date,
        dates = ID_data['date'],
        areas = ID_data['area'],
        weightsModel = density_estimator,
        cost_params = cost_params,
        gurobi_params = gurobi_params,
        K = K,
        rho = rho,
        print_status = False) for date in dates)
    
# Finalize
results = pd.concat(results).reset_index(drop=True)

# Save results
results.to_csv(path_results+"/"+model_name+"_K"+str(K)+".csv", sep=',', index=False)

# Time
print('Time:', dt.datetime.now().replace(microsecond=0) - start_time)  
print('>> Execution time:', np.around(time.time()-st_exec, 0), "seconds") 
print('>> CPU time:', np.around(time.process_time()-st_cpu, 0), "seconds")

Progress: 100%|██████████| 64/64 [01:05<00:00,  1.02s/it]

Time: 0:01:06
>> Execution time: 65.0 seconds
>> CPU time: 5.0 seconds


# Model: wSAA - RF

In [80]:
# Setup the model
model_setup = dict(

    # Model
    model_name = 'wSAA_RF',
        
    ## Density estimator
    density_estimator_params = dict(

        # Meta parameters
        model_params = {
            'random_state': 12345,
            'n_jobs': 4,
            'verbose': 0
        },

        # Tuning meta params
        tuning_params = {     
            #'probs': [0.005, 0.025, 0.165, 0.250, 0.500, 0.750, 0.835, 0.975, 0.995],
            'probs': sorted(list(set(np.concatenate([np.array([0.005, 0.025, 0.165, 0.25, 0.50, 0.835, 0.75, 0.975, 0.995]), 
                                                     np.arange(1, 100, 1) / 100])))),
            'nIter': 100,
            'random_state': 12345,
            'n_jobs': 16
        },    

       # Hyper params search grid
       hyper_params_grid = {
            'n_estimators': [100, 200, 300, 400, 500],
            'max_depth': [1, 3, 4, 5, 6, 7, 8, 9, 10],
            'min_samples_split': [10, 20, 40, 80, 160, 320],
            'min_samples_leaf': [10, 20, 40, 80, 160, 320],
            'max_features': [x/100 for x in range(5, 100, 5)],
            'max_leaf_nodes': [10, 20, 40, 80, 160, 320],
            'min_impurity_decrease': [0.0, 0.01, 0.02, 0.3, 0.04, 0.05],
            'bootstrap': [True],
            'max_samples': [0.80, 0.85, 0.90, 0.95, 1.00]
        },          
    )
)

# Make all model variables visible locally
locals().update(model_setup)

## Preprocessing

In [81]:
# Load data
Y_data = pd.read_csv('/home/fesc/dddex/PatientScheduling/Data/Y_data.csv')
X_data = pd.read_csv('/home/fesc/dddex/PatientScheduling/Data/X_data.csv')
ID_data = pd.read_csv('/home/fesc/dddex/PatientScheduling/Data/ID_data.csv')

In [82]:
# Train-test split
y_train = np.array(Y_data.loc[ID_data.train_test == 'train']).flatten()
X_train = np.array(X_data.loc[ID_data.train_test == 'train'])
ID_train = ID_data.loc[ID_data.train_test == 'train']

In [83]:
# CV folds
tscv = TimeSeriesSplit(n_splits=3)
cv_folds = tscv.split(range(len(ID_train)))
cv_folds = list(cv_folds)

In [84]:
# Initialize modules
weightsmodel = WeightsModel()
experiment = Experiment()

## Tune density estimator

In [85]:
# Update params
locals().update(density_estimator_params)

# Density estimator
density_estimator = RandomForestWSAA(**model_params)

# Tune density estimator
density_estimator = weightsmodel.tune_density_estimator(X_train, y_train, density_estimator, cv_folds, hyper_params_grid, 
                                                        tuning_params, random_search=True, print_time=True)

# Save
save = dump(density_estimator, path_models+'/density_estimator_'+model_name+'.joblib')

Time: 0:11:02
>> Execution time: 663.0 seconds
>> CPU time: 8.0 seconds


## Data-driven optimization

In [86]:
# Update params
locals().update(optimization_params)

# Load density estimator
density_estimator = load(path_models+'/density_estimator_'+model_name+'.joblib')

# Test dates
dates = pd.Series(list(set(ID_data.loc[ID_data.train_test == 'test', 'date']))).sort_values()

# Timer
start_time, st_exec, st_cpu = dt.datetime.now().replace(microsecond=0), time.time(), time.process_time()      
        
# For each date in the test horizon
with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(dates))) as progress_bar:
    results = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(
        X = np.array(X_data),
        y = np.array(Y_data),
        date = date,
        dates = ID_data['date'],
        areas = ID_data['area'],
        weightsModel = density_estimator,
        cost_params = cost_params,
        gurobi_params = gurobi_params,
        K = K,
        rho = rho,
        print_status = False) for date in dates)
    
# Finalize
results = pd.concat(results).reset_index(drop=True)

# Save results
results.to_csv(path_results+"/"+model_name+"_K"+str(K)+".csv", sep=',', index=False)

# Time
print('Time:', dt.datetime.now().replace(microsecond=0) - start_time)  
print('>> Execution time:', np.around(time.time()-st_exec, 0), "seconds") 
print('>> CPU time:', np.around(time.process_time()-st_cpu, 0), "seconds")

Progress: 100%|██████████| 64/64 [01:13<00:00,  1.16s/it]

Time: 0:01:14
>> Execution time: 74.0 seconds
>> CPU time: 6.0 seconds


# Model: SAA

In [87]:
# Setup the model
model_setup = dict(

    # Model
    model_name = 'SAA'

)

# Make all model variables visible locally
locals().update(model_setup)

## Preprocessing

In [88]:
# Load data
Y_data = pd.read_csv('/home/fesc/dddex/PatientScheduling/Data/Y_data.csv')
X_data = pd.read_csv('/home/fesc/dddex/PatientScheduling/Data/X_data.csv')
ID_data = pd.read_csv('/home/fesc/dddex/PatientScheduling/Data/ID_data.csv')

In [89]:
# Initialize modules
experiment = Experiment()

## Data-driven optimization

In [90]:
# Update params
locals().update(optimization_params)

# Test dates
dates = pd.Series(list(set(ID_data.loc[ID_data.train_test == 'test', 'date']))).sort_values()

# Timer
start_time, st_exec, st_cpu = dt.datetime.now().replace(microsecond=0), time.time(), time.process_time()      
        
# For each date in the test horizon
with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(dates))) as progress_bar:
    results = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(
        X = np.array(X_data),
        y = np.array(Y_data),
        date = date,
        dates = ID_data['date'],
        areas = ID_data['area'],
        weightsModel = SampleAverageApproximation(),
        cost_params = cost_params,
        gurobi_params = gurobi_params,
        K = K,
        rho = rho,
        print_status = False) for date in dates)
    
# Finalize
results = pd.concat(results).reset_index(drop=True)

# Save results
results.to_csv(path_results+"/"+model_name+"_K"+str(K)+".csv", sep=',', index=False)

# Time
print('Time:', dt.datetime.now().replace(microsecond=0) - start_time)  
print('>> Execution time:', np.around(time.time()-st_exec, 0), "seconds") 
print('>> CPU time:', np.around(time.process_time()-st_cpu, 0), "seconds")

Progress: 100%|██████████| 64/64 [00:57<00:00,  1.10it/s]

Time: 0:00:58
>> Execution time: 58.0 seconds
>> CPU time: 1.0 seconds


# Evaluate

In [133]:
# Load results
results_LSx_LGBM = pd.read_csv(path_results+'/LSx_LGBM_K'+str(K)+'.csv')
results_LSx_LGBM['model'] = 'LSx_LGBM'

results_LSx_LGBM_NN = pd.read_csv(path_results+'/LSx_LGBM_NN_K'+str(K)+'.csv')
results_LSx_LGBM_NN['model'] = 'LSx_LGBM_NN'

results_wSAA_RF = pd.read_csv(path_results+'/wSAA_RF_K'+str(K)+'.csv')
results_wSAA_RF['model'] = 'wSAA_RF'

results_SAA = pd.read_csv(path_results+'/SAA_K'+str(K)+'.csv')
results_SAA['model'] = 'SAA'

In [134]:
# Combine
results = pd.concat([
    results_LSx_LGBM[['model', 'date', 'area', 'CR', 'cost', 'overtime', 'waiting_time']],
    results_LSx_LGBM_NN[['model', 'date', 'area', 'CR', 'cost', 'overtime', 'waiting_time']],
    results_wSAA_RF[['model', 'date', 'area', 'CR', 'cost', 'overtime', 'waiting_time']]
])

results = pd.merge(
    results, 
    results_SAA[['date', 'area', 'CR', 'cost', 'overtime', 'waiting_time']],
    on=['date', 'area', 'CR'],
    suffixes=('', '_SAA')
)

In [135]:
# Coefficient of prescriptiveness
results['pq'] = results.cost / results.cost_SAA
results.loc[results.cost == results.cost_SAA, 'pq'] = 1

In [136]:
# Save results
results.to_csv(path_results+'/results_summary_K'+str(K)+'.csv', sep=',', index=False)

In [137]:
# Analysze (median) -- NEW
results.groupby(['CR', 'area', 'model']).agg(pq=('pq', np.median)).reset_index()

,CR,area,model,pq
0,0.1,Bereich_Bronchoskopie,LSx_LGBM,0.992547
1,0.1,Bereich_Bronchoskopie,LSx_LGBM_NN,0.995772
2,0.1,Bereich_Bronchoskopie,wSAA_RF,0.997998
3,0.1,Bereich_ERCP,LSx_LGBM,1.000000
4,0.1,Bereich_ERCP,LSx_LGBM_NN,1.000000
...,...,...,...,...
70,0.9,Bereich_Gastroskopie,LSx_LGBM_NN,0.881675
71,0.9,Bereich_Gastroskopie,wSAA_RF,0.906942
72,0.9,Bereich_Koloskopie,LSx_LGBM,0.947266
73,0.9,Bereich_Koloskopie,LSx_LGBM_NN,0.950888


In [104]:
# Analysze (median)
results.groupby(['CR', 'area', 'model']).agg(pq=('pq', np.median)).reset_index()

,CR,area,model,pq
0,0.1,Bereich_Bronchoskopie,LSx_LGBM,0.993647
1,0.1,Bereich_Bronchoskopie,LSx_LGBM_NN,0.992063
2,0.1,Bereich_Bronchoskopie,wSAA_RF,0.997998
3,0.1,Bereich_ERCP,LSx_LGBM,1.000000
4,0.1,Bereich_ERCP,LSx_LGBM_NN,1.000000
...,...,...,...,...
70,0.9,Bereich_Gastroskopie,LSx_LGBM_NN,0.886478
71,0.9,Bereich_Gastroskopie,wSAA_RF,0.906942
72,0.9,Bereich_Koloskopie,LSx_LGBM,0.941791
73,0.9,Bereich_Koloskopie,LSx_LGBM_NN,0.952113


In [110]:
# ... for selected CR
results.loc[results.CR.isin([0.10, 0.25, 0.50])].groupby(['CR', 'area', 'model']).agg(pq=('pq', np.median)).reset_index()

,CR,area,model,pq
0,0.10,Bereich_Bronchoskopie,LSx_LGBM,0.993647
1,0.10,Bereich_Bronchoskopie,LSx_LGBM_NN,0.992063
2,0.10,Bereich_Bronchoskopie,wSAA_RF,0.997998
3,0.10,Bereich_ERCP,LSx_LGBM,1.000000
4,0.10,Bereich_ERCP,LSx_LGBM_NN,1.000000
5,0.10,Bereich_ERCP,wSAA_RF,1.000000
6,0.10,Bereich_Endosonographie,LSx_LGBM,1.000000
7,0.10,Bereich_Endosonographie,LSx_LGBM_NN,1.000000
8,0.10,Bereich_Endosonographie,wSAA_RF,1.000000
9,0.10,Bereich_Gastroskopie,LSx_LGBM,0.957436


In [ ]:
# ... for selected CR and area
results.loc[
    results.CR.isin([0.10, 0.25, 0.50]) & 
    total.area.isin(['Bereich_Gastroskopie', 'Bereich_Koloskopie'])].groupby(['CR', 'area', 'model']).agg(
    pq=('pq', np.median)).reset_index()

In [141]:
# Analysze (total) --- NEW
total = results.groupby(['CR', 'area', 'model']).agg(total_cost=('cost', sum), total_cost_SAA=('cost_SAA', sum)).reset_index()
total['pq'] = total.total_cost / total.total_cost_SAA
total

,CR,area,model,total_cost,total_cost_SAA,pq
0,0.1,Bereich_Bronchoskopie,LSx_LGBM,9784.0,9788.0,0.999591
1,0.1,Bereich_Bronchoskopie,LSx_LGBM_NN,9834.0,9788.0,1.004700
2,0.1,Bereich_Bronchoskopie,wSAA_RF,9822.0,9788.0,1.003474
3,0.1,Bereich_ERCP,LSx_LGBM,7771.0,7925.0,0.980568
4,0.1,Bereich_ERCP,LSx_LGBM_NN,7789.0,7925.0,0.982839
...,...,...,...,...,...,...
70,0.9,Bereich_Gastroskopie,LSx_LGBM_NN,136646.0,168066.0,0.813050
71,0.9,Bereich_Gastroskopie,wSAA_RF,143155.0,168066.0,0.851778
72,0.9,Bereich_Koloskopie,LSx_LGBM,42863.0,45284.0,0.946537
73,0.9,Bereich_Koloskopie,LSx_LGBM_NN,42200.0,45284.0,0.931896


In [108]:
# Analysze (total)
total = results.groupby(['CR', 'area', 'model']).agg(total_cost=('cost', sum), total_cost_SAA=('cost_SAA', sum)).reset_index()
total['pq'] = total.total_cost / total.total_cost_SAA
total

,CR,area,model,total_cost,total_cost_SAA,pq
0,0.1,Bereich_Bronchoskopie,LSx_LGBM,9860.0,9788.0,1.007356
1,0.1,Bereich_Bronchoskopie,LSx_LGBM_NN,9815.0,9788.0,1.002758
2,0.1,Bereich_Bronchoskopie,wSAA_RF,9822.0,9788.0,1.003474
3,0.1,Bereich_ERCP,LSx_LGBM,7852.0,7925.0,0.990789
4,0.1,Bereich_ERCP,LSx_LGBM_NN,7787.0,7925.0,0.982587
...,...,...,...,...,...,...
70,0.9,Bereich_Gastroskopie,LSx_LGBM_NN,137003.0,168066.0,0.815174
71,0.9,Bereich_Gastroskopie,wSAA_RF,143155.0,168066.0,0.851778
72,0.9,Bereich_Koloskopie,LSx_LGBM,40797.0,45284.0,0.900914
73,0.9,Bereich_Koloskopie,LSx_LGBM_NN,42010.0,45284.0,0.927701


In [111]:
# ... for selected CR
total.loc[total.CR.isin([0.10, 0.25, 0.50])]

,CR,area,model,total_cost,total_cost_SAA,pq
0,0.10,Bereich_Bronchoskopie,LSx_LGBM,9860.0,9788.0,1.007356
1,0.10,Bereich_Bronchoskopie,LSx_LGBM_NN,9815.0,9788.0,1.002758
2,0.10,Bereich_Bronchoskopie,wSAA_RF,9822.0,9788.0,1.003474
3,0.10,Bereich_ERCP,LSx_LGBM,7852.0,7925.0,0.990789
4,0.10,Bereich_ERCP,LSx_LGBM_NN,7787.0,7925.0,0.982587
5,0.10,Bereich_ERCP,wSAA_RF,7891.0,7925.0,0.995710
6,0.10,Bereich_Endosonographie,LSx_LGBM,4298.0,4441.0,0.967800
7,0.10,Bereich_Endosonographie,LSx_LGBM_NN,4286.0,4441.0,0.965098
8,0.10,Bereich_Endosonographie,wSAA_RF,4274.0,4441.0,0.962396
9,0.10,Bereich_Gastroskopie,LSx_LGBM,50062.0,54748.0,0.914408


In [142]:
# ... for selected CR and area
total.loc[total.CR.isin([0.10, 0.25, 0.50]) & total.area.isin(['Bereich_Gastroskopie', 'Bereich_Koloskopie'])]

,CR,area,model,total_cost,total_cost_SAA,pq
9,0.10,Bereich_Gastroskopie,LSx_LGBM,50066.0,54748.0,0.914481
10,0.10,Bereich_Gastroskopie,LSx_LGBM_NN,49883.0,54748.0,0.911138
11,0.10,Bereich_Gastroskopie,wSAA_RF,50371.0,54748.0,0.920052
12,0.10,Bereich_Koloskopie,LSx_LGBM,30072.0,30984.0,0.970565
13,0.10,Bereich_Koloskopie,LSx_LGBM_NN,29703.0,30984.0,0.958656
14,0.10,Bereich_Koloskopie,wSAA_RF,29771.0,30984.0,0.960851
24,0.25,Bereich_Gastroskopie,LSx_LGBM,27275.0,31601.0,0.863106
25,0.25,Bereich_Gastroskopie,LSx_LGBM_NN,27160.0,31601.0,0.859466
26,0.25,Bereich_Gastroskopie,wSAA_RF,27833.0,31601.0,0.880763
27,0.25,Bereich_Koloskopie,LSx_LGBM,14122.0,14677.0,0.962186


In [140]:
# ... for selected CR and area --- NEW
total.loc[total.CR.isin([0.10, 0.25, 0.50]) & total.area.isin(['Bereich_Gastroskopie', 'Bereich_Koloskopie'])]

,CR,area,model,total_cost,total_cost_SAA,pq
9,0.10,Bereich_Gastroskopie,LSx_LGBM,50066.0,54748.0,0.914481
10,0.10,Bereich_Gastroskopie,LSx_LGBM_NN,49883.0,54748.0,0.911138
11,0.10,Bereich_Gastroskopie,wSAA_RF,50371.0,54748.0,0.920052
12,0.10,Bereich_Koloskopie,LSx_LGBM,30072.0,30984.0,0.970565
13,0.10,Bereich_Koloskopie,LSx_LGBM_NN,29703.0,30984.0,0.958656
14,0.10,Bereich_Koloskopie,wSAA_RF,29771.0,30984.0,0.960851
24,0.25,Bereich_Gastroskopie,LSx_LGBM,27275.0,31601.0,0.863106
25,0.25,Bereich_Gastroskopie,LSx_LGBM_NN,27160.0,31601.0,0.859466
26,0.25,Bereich_Gastroskopie,wSAA_RF,27833.0,31601.0,0.880763
27,0.25,Bereich_Koloskopie,LSx_LGBM,14122.0,14677.0,0.962186


# Debugging

In [ ]:
#### ... 
def predict_quantiles(X, y, date, dates, areas, weightsModel = None, 
             quantiles = [0.005, 0.025, 0.165, 0.250, 0.500, 0.750, 0.835, 0.975, 0.995],
             print_status = False):



    """

    ...

    Arguments:

        X: ...
        y: ...
        date: ...
        dates: ...
        areas: ...
        weightsModel = None: ...
        quantiles = [0.005, 0.025, 0.165, 0.250, 0.500, 0.750, 0.835, 0.900, 0.975, 0.995]: ...
        print_status = True: ...

    Returns:

        results(pd.DataFrame): ...


    """

    # Train-test split
    y_train, y_test = y[dates < date].flatten(), y[dates == date].flatten()
    X_train, X_test = X[dates < date], X[dates == date]
    dates_train, dates_test = dates[dates < date], dates[dates == date]
    areas_train, areas_test = areas[dates < date], areas[dates == date]

    # Fit weights model
    weightsModel.fit(X_train, y_train)

    # Initialize
    results = {}

    # For each area 
    for area in list(set(areas_test)):

        # Select test data for current area
        y_test_ = y_test[area == areas_test]
        X_test_ = X_test[area == areas_test]

        # Fit SAA on data of current area
        SAA = SampleAverageApproximation()
        SAA.fit(y_train[area == areas_train])

        # Predict quantiles for each patient
        q_hat = weightsModel.predict(X_test_, probs=quantiles, outputAsDf=False)
        q_hat_saa = SAA.predict(X_test_, probs=quantiles, outputAsDf=False)

        # Add to results
        results[area] = pd.merge(
            left = pd.concat(pd.DataFrame({'prob': p, 'q_hat': q}) for p, q in q_hat.items()).reset_index(names='j'),
            right = pd.concat(pd.DataFrame({'prob': p, 'q_hat_saa': q, 'y': y_test_}) for p, q in q_hat_saa.items()).reset_index(names='j'),
            on = ['prob', 'j']
        )
        
    # Finalize
    results = pd.concat(results).reset_index(names=['area', '']).drop(columns='')

    return results

In [ ]:
# Update params
locals().update(optimization_params)

# Test dates
dates = pd.Series(list(set(ID_data.loc[ID_data.train_test == 'test', 'date']))).sort_values()

# Timer
start_time, st_exec, st_cpu = dt.datetime.now().replace(microsecond=0), time.time(), time.process_time()      
        
# For each date in the test horizon
with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(dates))) as progress_bar:
    results = Parallel(n_jobs=n_jobs)(delayed(predict_quantiles)(
        X = np.array(X_data),
        y = np.array(Y_data),
        date = date,
        dates = ID_data['date'],
        areas = ID_data['area'],
        weightsModel = density_estimator,
        quantiles = [0.005, 0.025, 0.165, 0.250, 0.500, 0.750, 0.835, 0.975, 0.995],
        print_status = False) for date in dates)

# Finalize
results = pd.concat(results, keys=dates).reset_index(names=['date', '']).drop(columns='')

# Time
print('Time:', dt.datetime.now().replace(microsecond=0) - start_time)  
print('>> Execution time:', np.around(time.time()-st_exec, 0), "seconds") 
print('>> CPU time:', np.around(time.process_time()-st_cpu, 0), "seconds")

In [ ]:
# Scaled Pinball Loss
def scaled_pinball_loss(p, q, q_saa, y, **kwargs):

    """

    ...

    """
    q = np.array(q).flatten()
    q_saa = np.array(q_saa).flatten()
    y = np.array(y).flatten()
    

    # Pinball Loss Model
    pl = np.mean((y - q) * p * (q <= y) + (q - y) * (1 - p) * (q > y))

    # Pinball Loss SAA
    pl_saa = np.mean((y - q_saa) * p * (q_saa <= y) + (q_saa - y) * (1 - p) * (q_saa > y))

    # Scaled Pinball Loss
    with np.errstate(divide='ignore'):
        spl = (pl == pl_saa) * 1.0 + (pl != pl_saa) * (pl / pl_saa)

    return spl

In [ ]:
spl = results.groupby(['area', 'prob']).apply(
    
    lambda df: pd.Series({'spl': scaled_pinball_loss(p=df.prob, q=df.q_hat, q_saa=df.q_hat_saa, y=df.y)
    
    })

).reset_index()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Number of cases per day by treatment
plotData = spl.groupby(['area'])

# Plot
fig, ax = plt.subplots()
fig.set_size_inches(10, 5)

for area, d in plotData:
    ax.plot(d['prob'], d['spl'], marker='', linestyle='-', ms=2, linewidth=2, label=area)
plt.axhline(y = 1, color = 'grey', linestyle = '--', linewidth=1)
ax.legend()
#ax.get_xaxis().set_visible(False)
plt.show()

## LSx LGBM

In [ ]:
# Setup the model
model_setup = dict(

    # Model
    model_name = 'LSx_LGBM',
        
    ## Point estimator
    point_estimator_params = dict(

        # Meta parameters
        model_params = {
            'random_state': 12345,
            'n_jobs': 4,
            'verbose': -1
        },

        # Tuning meta params
        tuning_params = {     
            'n_iter': 200,
            'scoring': {'MSE': 'neg_mean_squared_error'},
            'return_train_score': True,
            'refit': 'MSE',
            'random_state': 12345,
            'n_jobs': 8,
            'verbose': 0
        },    

        # Hyper params search grid
       hyper_params_grid = {
            'num_leaves': [x for x in range(5, 500, 5)],
            'max_depth': [-1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            'min_child_samples': [x for x in range(5, 500, 5)],
            'learning_rate': [x/100 for x in range(5, 25, 5)],
            'n_estimators': [100, 200, 300, 400, 500],
            'subsample': [x/100 for x in range(5, 100, 5)],
            'colsample_bytree': [x/100 for x in range(5, 100, 5)]
        },  
    
    
    ),
          
 
    ## Density estimator
    density_estimator_params = dict(
    
        # Meta parameters
        model_params = {
            'weightsByDistance': False
        },

        # Tuning meta params
        tuning_params = {     
            #'probs': [0.005, 0.025, 0.165, 0.250, 0.500, 0.750, 0.835, 0.975, 0.995],
            'probs': sorted(list(set(np.concatenate([np.array([0.005, 0.025, 0.165, 0.25, 0.50, 0.835, 0.75, 0.975, 0.995]), 
                                                     np.arange(1, 100, 1) / 100])))),
            'n_jobs': 8,
        },    

        # Hyper params search grid
        hyper_params_grid = {
            'binSize': [x for x in range(10, 500, 10)]
        },
        
    )
)

# Make all model variables visible locally
locals().update(model_setup)

## Preprocessing

In [ ]:
# Load data
Y_data = pd.read_csv('/home/fesc/dddex/PatientScheduling/Data/Y_data.csv')
X_data = pd.read_csv('/home/fesc/dddex/PatientScheduling/Data/X_data.csv')
ID_data = pd.read_csv('/home/fesc/dddex/PatientScheduling/Data/ID_data.csv')

In [ ]:
# Train-test split
y_train = np.array(Y_data.loc[ID_data.train_test == 'train']).flatten()
X_train = np.array(X_data.loc[ID_data.train_test == 'train'])
ID_train = ID_data.loc[ID_data.train_test == 'train']

In [ ]:
# CV folds
tscv = TimeSeriesSplit(n_splits=3)
cv_folds = tscv.split(range(len(ID_train)))
cv_folds = list(cv_folds)

In [ ]:
# Initialize modules
weightsmodel = WeightsModel()
experiment = Experiment()

## Tune point estimator

In [ ]:
# Update params
locals().update(point_estimator_params)

# Point estimator
point_estimator = LGBMRegressor(**model_params)

# Tune point estimator
point_estimator = weightsmodel.tune_point_estimator(X_train, y_train, point_estimator, cv_folds, hyper_params_grid, 
                                                    tuning_params, random_search=True, print_time=True)

## Tune density estimator

In [ ]:
# Update params
locals().update(density_estimator_params)

# Density estimator
density_estimator = LevelSetKDEx(estimator = point_estimator, **model_params)

# Tune density estimator
density_estimator = weightsmodel.tune_density_estimator(X_train, y_train, density_estimator, cv_folds, hyper_params_grid, 
                                                        tuning_params, random_search=False, print_time=True)

In [ ]:
#density_estimator = density_estimator.set_params(binSize=5)

## Data-driven optimization

In [ ]:
# Update params
locals().update(optimization_params)

# Test dates
dates = pd.Series(list(set(ID_data.loc[ID_data.train_test == 'test', 'date']))).sort_values()

date = min(dates)

results = experiment.run(
    X = np.array(X_data),
    y = np.array(Y_data),
    date = date,
    dates = ID_data['date'],
    areas = ID_data['area'],
    weightsModel = density_estimator,
    cost_params = cost_params,
    gurobi_params = gurobi_params,
    K = K,
    alpha = alpha,
    print_status = False)

In [ ]:
## in depth

# Update params
locals().update(optimization_params)

# Test dates
dates = pd.Series(list(set(ID_data.loc[ID_data.train_test == 'test', 'date']))).sort_values()

date = min(dates)


In [ ]:
X = np.array(X_data)
y = np.array(Y_data)
date = date
dates = ID_data['date']
areas = ID_data['area']

#weightsModel = density_estimator

weightsModel = SampleAverageApproximation()


cost_params = cost_params
gurobi_params = gurobi_params
K = 10*3
alpha = 2
print_status = False

In [ ]:
# Train-test split
y_train, y_test = y[dates < date].flatten(), y[dates == date].flatten()
X_train, X_test = X[dates < date], X[dates == date]
dates_train, dates_test = dates[dates < date], dates[dates == date]
areas_train, areas_test = areas[dates < date], areas[dates == date]

# Get time budget per area
hist_durations = pd.DataFrame({
    'duration': y_train, 
    'area': areas_train,
}).groupby('area').agg(
    median_duration=('duration', np.median)
).reset_index()

hist_durations = dict(zip(hist_durations.area, hist_durations.median_duration))

In [ ]:
weightsModel

In [ ]:
# LSx and wSAA
if not str(type(weightsModel)) == "<class 'dddex.wSAA.SampleAverageApproximation'>":

    # Fit weights model
    weightsModel.refitPointEstimator(X=X_train, y=y_train)
    weightsModel.fit(X=X_train, y=y_train)

In [ ]:
# Initialize
results = pd.DataFrame()

area = 'Bereich_Gastroskopie'


# Initialize
results_ = {}

# Progress
if print_status:
    print('=====================================================================================================')
    print('# Area:',area)

# Timer
weights_st_exec = time.time()
weights_st_cpu = time.process_time() 

# Select test data for current area
y_test_ = y_test[area == areas_test]
X_test_ = X_test[area == areas_test]

# Number of patient cases to schedule
M = len(y_test_)

# Set random sequence of patient cases
#patient_sequence = np.arange(M)
#np.random.shuffle(patient_sequence)

#y_test_ = y_test_[patient_sequence]
#X_test_ = X_test_[patient_sequence]

# Set time budget (based on median duration per area x number of cases on test day)
T = hist_durations[area] * M * alpha

# SAA
if str(type(weightsModel)) == "<class 'dddex.wSAA.SampleAverageApproximation'>":

    # Fit SAA on data of current area
    weightsModel.fit(y_train[area == areas_train])

# Get estimated conditional distribution
conditionalDistribution = weightsModel.getWeights(X_test_, outputType='summarized')

# Timer
weights_exec_time_sec = time.time()-weights_st_exec
weights_cpu_time_sec = time.process_time()-weights_st_cpu

In [ ]:
# Generate K scenarios
scenarios = []

# Timer
scenarios_st_exec = time.time()
scenarios_st_cpu = time.process_time() 

# Draw
for k in range(K):    

    # For each patient
    scenario = []
    for j in range(M):

        # Weighted samples
        weights = conditionalDistribution[j][0]
        samples = conditionalDistribution[j][1]

        # Add scenario for patient j
        scenario += [np.random.choice(samples.flatten(), p=weights.flatten())]

    # Add scenarios
    scenarios += [scenario]

# Scenarios
scenarios = np.array(scenarios)   

# Timer
scenarios_exec_time_sec = time.time()-scenarios_st_exec
scenarios_cpu_time_sec = time.process_time()-scenarios_st_cpu

In [ ]:
from DataDrivenPatientScheduling import PatientScheduling

In [ ]:
cost_params_ = cost_params[2]

# Progress
if print_status:
    print('## Cost param setting:',cost_params_)

# Timer
opt_st_exec = time.time()
opt_st_cpu = time.process_time() 

# Optimization
CR, c_waiting_time, c_overtime = cost_params_['CR'], cost_params_['c_waiting_time'], cost_params_['c_overtime']

# Initialize optimization model
ddps = PatientScheduling(c_waitingtime=c_waiting_time, c_overtime=c_overtime, T=T, **gurobi_params)

# Set up optimization model with scenarios
ddps.create(z=scenarios)

# Solve
schedule, times, status, solutions, gap = ddps.optimize()

# Waiting time
scheduled_start, scheduled_duration, actual_duration, waiting_time = [], [], [], []    

for j in range(M):

    scheduled_start += [schedule[j]]
    scheduled_duration += [times[j]]
    actual_duration += list(y[j])
    waiting_time += [0] if j == 0 else [max([0]+[waiting_time[j-1]+actual_duration[j-1]-scheduled_duration[j-1]])]

# Overtime (using last j)
overtime = max([0]+[waiting_time[j]+actual_duration[j]-scheduled_duration[j]])

# Cost
cost = c_waiting_time * sum(waiting_time) + c_overtime * overtime

# Timer
optimization_exec_time_sec = time.time()-opt_st_exec
optimization_cpu_time_sec = time.process_time()-opt_st_cpu

# Store results
result = {

    'date': date,
    'area': area,
    'n_patients': M,
    'n_scenarios': K,
    'CR': CR,
    'c_waiting_time': c_waiting_time,
    'c_overtime': c_overtime,
    'historical_time_budget': hist_durations[area] * M,
    'time_budget_multiplier': alpha,
    'total_time_budget': T,
    'waiting_time': sum(waiting_time),
    'overtime': overtime,
    'cost': cost,
    'scheduled_total_durations': sum(scheduled_duration),
    'actual_total_durations': sum(actual_duration),
    'optimization_exec_time_sec': optimization_exec_time_sec,
    'optimization_cpu_time_sec': optimization_cpu_time_sec,
    'scenarios_exec_time_sec': scenarios_exec_time_sec,
    'scenarios_cpu_time_sec': scenarios_cpu_time_sec,
    'weights_exec_time_sec': weights_exec_time_sec,
    'weights_cpu_time_sec': weights_cpu_time_sec

}



In [ ]:
result

In [ ]:
#SAA
result

In [ ]:
#SAA
result

# Optimization problem step-by-step

In [ ]:
## in depth

# Update params
locals().update(optimization_params)

# Test dates
dates = pd.Series(list(set(ID_data.loc[ID_data.train_test == 'test', 'date']))).sort_values()

date = min(dates)


In [ ]:
X = np.array(X_data)
y = np.array(Y_data)
date = date
dates = ID_data['date']
areas = ID_data['area']

weightsModel = density_estimator

#weightsModel = SampleAverageApproximation()


cost_params = cost_params
gurobi_params = gurobi_params
K = 10*2
alpha = 1
print_status = False

In [ ]:
# Train-test split
y_train, y_test = y[dates < date].flatten(), y[dates == date].flatten()
X_train, X_test = X[dates < date], X[dates == date]
dates_train, dates_test = dates[dates < date], dates[dates == date]
areas_train, areas_test = areas[dates < date], areas[dates == date]

# Get time budget per area
hist_durations = pd.DataFrame({
    'duration': y_train, 
    'area': areas_train,
}).groupby('area').agg(
    median_duration=('duration', np.median)
).reset_index()

hist_durations = dict(zip(hist_durations.area, hist_durations.median_duration))

In [ ]:
# LSx and wSAA
if not str(type(weightsModel)) == "<class 'dddex.wSAA.SampleAverageApproximation'>":

    # Fit weights model
    weightsModel.refitPointEstimator(X=X_train, y=y_train)
    weightsModel.fit(X=X_train, y=y_train)

In [ ]:
# Initialize
results = pd.DataFrame()

area = 'Bereich_Gastroskopie'


# Initialize
results_ = {}

# Progress
if print_status:
    print('=====================================================================================================')
    print('# Area:',area)

# Timer
weights_st_exec = time.time()
weights_st_cpu = time.process_time() 

# Select test data for current area
y_test_ = y_test[area == areas_test]
X_test_ = X_test[area == areas_test]

# Number of patient cases to schedule
M = len(y_test_)

# Set random sequence of patient cases
#patient_sequence = np.arange(M)
#np.random.shuffle(patient_sequence)

#y_test_ = y_test_[patient_sequence]
#X_test_ = X_test_[patient_sequence]

# Set time budget (based on median duration per area x number of cases on test day)
T = hist_durations[area] * M * alpha

# SAA
if str(type(weightsModel)) == "<class 'dddex.wSAA.SampleAverageApproximation'>":

    # Fit SAA on data of current area
    weightsModel.fit(y_train[area == areas_train])

# Get estimated conditional distribution
conditionalDistribution = weightsModel.getWeights(X_test_, outputType='summarized')

# Timer
weights_exec_time_sec = time.time()-weights_st_exec
weights_cpu_time_sec = time.process_time()-weights_st_cpu

In [ ]:
# Generate K scenarios
scenarios = []

# Timer
scenarios_st_exec = time.time()
scenarios_st_cpu = time.process_time() 

# Draw
for k in range(K):    

    # For each patient
    scenario = []
    for j in range(M):

        # Weighted samples
        weights = conditionalDistribution[j][0]
        samples = conditionalDistribution[j][1]

        # Add scenario for patient j
        scenario += [np.random.choice(samples.flatten(), p=weights.flatten())]

    # Add scenarios
    scenarios += [scenario]

# Scenarios
scenarios = np.array(scenarios)   

# Timer
scenarios_exec_time_sec = time.time()-scenarios_st_exec
scenarios_cpu_time_sec = time.process_time()-scenarios_st_cpu

In [ ]:
# Init
z = scenarios

# Number of scenarios
K = z.shape[0]

# Number of patients
M = z.shape[1]

# Problem parameters
c_waitingtime = 9
c_overtime = 1
T = T

m = gp.Model()

In [ ]:
## Create decision variables

# Primary decision variable (sequential time budget)
x = m.addVars(M, vtype='I', lb=0, name='x')

# Auxiary decision variable (waiting time)
w = m.addVars(K, M, vtype='I', lb=0, name='w')        

# Auxiary decision variable (overtime)
l = m.addVars(K, vtype='I', lb=0, name='l') 

In [ ]:


## Set objective function 
OBJ = m.setObjective(

    # Sample average
    gp.quicksum(

        1/K * (                                         

            # Waiting time
            c_waitingtime * gp.quicksum(w[k,j] for j in range(1, M)) + 

            # Overtime
            c_overtime * l[k]


        ) for k in range(K)),        

    # min
    GRB.MINIMIZE
)


    


In [ ]:
## Set constraints

# Waiting times
C1 = m.addConstrs(

    w[k,j] + z[k,j] - x[j] <= w[k,j+1]

    for j in range(M-1)
    for k in range(K)

)  

# Overtime
C2 = m.addConstrs(

    w[k,M-1] + z[k,M-1] - x[M-1] <= l[k]

    for k in range(K)

)    

# Total time budget
if not T is None:

    C3 = m.addConstr(

        gp.quicksum(x[j] for j in range(M)) <= T

    ) 
    
    
# Waiting time of first patient is always zero
C4 = m.addConstrs(

    w[k,0] == 0

    for k in range(K)

)

In [ ]:
# Set Gurobi meta params
m.setParam('LogToConsole', 1)
m.setParam('Threads', 32)

# Optimize
m.optimize()     

# Solution
if m.SolCount > 0:

    times = [var.xn for var in m.getVars() if 'x' in var.VarName]

else:

    times = np.nan

# Schedule (start times created using cumulative time budgets, i.e., 0, x_{1}, x_{1} + x_{2}, ..., x_{1} + ... + x_{M-1})
schedule = np.cumsum([0] + times[0:(len(times)-1)])


# Solution meta data
status = m.status
solutions = m.SolCount
gap = m.MIPGap

In [ ]:
# Waiting time
scheduled_start, scheduled_duration, actual_duration, waiting_time = [], [], [], []    

for j in range(M):

    scheduled_start += [schedule[j]]
    scheduled_duration += [times[j]]
    actual_duration += [y_test_[j]]
    waiting_time += [0] if j == 0 else [max([0]+[waiting_time[j-1]+actual_duration[j-1]-scheduled_duration[j-1]])]

# Overtime (using last j)
overtime = max([0]+[waiting_time[j]+actual_duration[j]-scheduled_duration[j]])

# Cost
cost = c_waiting_time * sum(waiting_time) + c_overtime * overtime

In [ ]:
cost